In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils import configs, extract, join_data, transform

import json

In [46]:
extract.extract()
transform.transform()

In [50]:
join_data.join_data()

In [24]:

def header_fn(data):
    """
    Extract misc player information not included in season/gamlog stats
    """
    header_raw = data['header'][2:]

    # get position and team, if no team exists team = ""
    position_team = header_raw.pop(0).split(", ")
    position_team.append("")
    (pos, team) = position_team[:2]

    # format string for key value structure
    draft_class_replace_tuples = (" ", "_"), (":_",": "), ("(", ""), (")", "")
    for t in draft_class_replace_tuples:
        header_raw[0] = header_raw[0].replace(*t)
    header_raw.append(header_raw.pop().replace("  ", " "))

    # add pos and team in key value structure
    header_raw.append(f'pos: {pos.lower()}')
    header_raw.append(f'team: {team.lower()}')

    # seperate keys from values
    header_key_values = [item.replace(": ", " ").split(" ") for item in header_raw]

    # populate dict with keys and values
    header_dict_collection = {}
    for row in header_key_values:
        for i, item in enumerate(row):
            if i % 2:
                header_dict_collection[key] = item.lower()
            else:
                key = item.lower()

    return header_dict_collection

In [25]:
def group(data):
    """
    Separate raw data into groups defined in configs.table_names
    header group accessed by index (0th and 2nd row of raw data)
    other groups accessed by string search
    """
    table_names = configs.table_names
    
    player_dict = {}
    player_dict[table_names[0]] = data[0] + data[2]

    for row in data:
        for string in table_names:
            if row[0] == string:
                string_key = string.replace(" ", "_").lower()
                player_dict[string_key] = row
    
    return player_dict

In [49]:
master_raw_data = json.load(open(configs.raw_loc, 'rb'))
raw_data = {k: v['data'] for k, v in master_raw_data.items()}

for key in raw_data:
    raw_data[key] = raw_data[key].replace("\n\n\n", "\n\n")
    raw_data[key] = raw_data[key].split("\n\n")

for key in raw_data:
    data_split = []
    for item in raw_data[key]:
        data_split.append(item.split("\n"))
    raw_data[key] = data_split

for k, data in raw_data.items():
    raw_data[k] = group(data)
    # raw_data[k] = header_fn(grouped_data)

In [99]:
for player, data in raw_data.items():

    header = {}
    raw_header = data['header']

    header['pos'], header['team'] = raw_header[2].split(", ")
    for item in raw_header[3:]:
        if ":" in item:
            split_item = item.split(": ")
            key = split_item[0]
            if key == "College":
                header[key] = split_item[1]
            if key == "Draft":
                header[key] = split_item[1].replace(" ", "_").replace("(", "").replace(")", "")
            if key in ["Ht", "DOB", "Age"]:
                item = item.replace(": ", ":").replace("  ", " ").split(" ")
                for subitem in item:
                    if ":" in subitem:
                        key, val = subitem.split(":")
                        header[key] = val

    print(player, header.keys())

Patrick Mahomes dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Josh Allen dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Carson Wentz dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Jalen Hurts dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Justin Herbert dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Lamar Jackson dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Joe Burrow dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Marcus Mariota dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Jameis Winston dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Ryan Tannehill dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Kirk Cousins dict_keys(['pos', 'team', 'Draft', 'College', 'Ht', 'Wt', 'DOB', 'Age'])
Baker Mayfield dict_keys(['pos', 'team', 'Draft